- Импорт нужных модулей:

In [1]:
import requests
from bs4 import BeautifulSoup
import pprint
import pandas as pd
import re

Запрос:

In [2]:
url = 'https://stihi.ru/'
response = requests.get(url)
response.status_code

200

Полный текст запроса:

In [3]:
full = response.text

- колонка с рекомендациями
- верхнее меню

In [4]:
soup = BeautifulSoup(full, 'html.parser')
my_column = soup.find_all('div', {'class': 'col1'})
head = soup.find_all('ul', {'class': 'headmenu'})

- ссылки из верхнего меню

In [5]:
head_soup = BeautifulSoup(str(head), 'html.parser')
links = head_soup.find_all('a', {'class': None})

In [6]:
for l in links:
    print (l)

<a href="/authors/">Авторы</a>
<a href="/poems/">Произведения</a>
<a href="/board/">Рецензии</a>
<a href="/search.html">Поиск</a>
<a href="https://shop.stihi.ru/">Магазин</a>
<a href="https://o.stihi.ru/"><nobr>О портале</nobr></a>
<a href="/login/">Вход для авторов</a>


ищем нужную ссылку на рекомендуемые произведения:

In [7]:
soup = BeautifulSoup(str(my_column), 'html.parser')
rec = soup.find_all('p', {'class': 'toptext'})
rec_soup = BeautifulSoup(str(rec), 'html.parser')
my_link = rec_soup.find_all('a', {'class': 'topref'})
for t in my_link:
    if 'произведения' in t.text:
        my_href = t.attrs['href']
    else:
        pass
new_url = url+my_href
new_url

'https://stihi.ru//editor/'

In [8]:
response = requests.get(new_url)
response.status_code

200

In [9]:
full_poems = response.text

Подготовка словаря, куда будут записаны данные:

In [10]:
my_data = {'Автор': [], 'Название': [], 'Год публ.': [],
           'Месяц публ.': [], 'День публ.': [], 'Время публ.': []}

Функция, которая добавляет в словарь `my_dict` данные `link` по ключу `part`:

In [11]:
def plus(part, link, my_dict):
    my_dict[part].append(link)

`make_data_one` -- функция, которая ищет автора или заголовок и добавляет его в словарь

`make_data_two` -- функция, которая ищет год, месяц, день или время и добавляет его в словарь

In [12]:
def make_data_one(name, my_type, my_class, data, new_name):
    name = link_soup.find(my_type, {'class': my_class})
    if name != None:
        plus(new_name, name.text, data)
        #print(name.text)
        
def make_data_two(name, data, new_name, num):
    name = link_soup.find('small')
    if name != None:
        good_text = re.split('\.| |-', name.text)
        plus(new_name, good_text[num], data)

Теперь эти функции применяются к нужной части сайта:

In [13]:
poems_soup = BeautifulSoup(full_poems, 'html.parser')
my_link = poems_soup.find_all('li')
for l in my_link:
    link_soup = BeautifulSoup(str(l), 'html.parser')
    make_data_one('author', 'a', 'authorlink', my_data, 'Автор')
    make_data_one('my_title', 'a', 'poemlink', my_data, 'Название')
    make_data_two('year', my_data, 'Год публ.', 4)
    make_data_two('month', my_data, 'Месяц публ.', 3)
    make_data_two('day', my_data, 'День публ.', 2)
    make_data_two('time', my_data, 'Время публ.', 5)

Шапка датафрейма:

In [14]:
my_df = pd.DataFrame(my_data)
my_df.head()

,Автор,Название,Год публ.,Месяц публ.,День публ.,Время публ.
0,Соня Пащевская,Вечер октября,2020,10,11,11:24
1,Сергей Кривонос,Кот Семен,2020,10,11,09:52
2,Марианна Боровкова,Это голуби,2020,10,11,09:52
3,Таня Зачёсова,"Осень, октябрь и огненно-рыжий в моде",2020,10,11,09:46
4,Юлия Зазимко,Сказочка,2020,10,11,09:44


Отсортированные данные:

In [15]:
my_df.sort_values(by='Год публ.').sort_values(by='Месяц публ.').sort_values(by='День публ.')

,Автор,Название,Год публ.,Месяц публ.,День публ.,Время публ.
28,Таня Зачёсова,"Я словно лист, что не желает падать",2020,10,07,15:31
22,Мария Фроловская,Маленький посёлок на побережье,2020,10,07,11:17
21,Сергей Хомутов Рыбинск,Бывший друг,2020,10,07,11:20
20,Клавдия Смирягина Дмитриева,Кровь осени пульсирует в висках,2020,10,07,11:21
19,Андрей Войтов,Хрупкие наивные мечты,2020,10,07,11:24
18,Ирина Молочкова,Всё прозрачней,2020,10,07,11:27
17,Игорь Гуревич,Минута осени,2020,10,07,11:30
16,Сергей Вит Берсенев,Потеряться,2020,10,07,11:34
15,Лада Миллер,За озером вступили духовые,2020,10,07,11:35
27,Евгений Захарченко 2,Свидание,2020,10,07,15:33
